In [1]:
from tkinter import *
import time
import random
import heapq
from math import sqrt

In [2]:
#please note that gui code and priority queue were taken from github with significant change and generalization.

#initilizing grid dimensions menu
root = Tk()
root.geometry("300x300")
root.title("PA1")
frame = Frame(root, width=300, height= 300)
frame.pack()
frame.place()
rows_label = Label(frame,text = "Rows")
cols_label = Label(frame,text = "Columns")
rows = 0
cols = 0

#prompt for dimensions
rows_label.grid(row=0,column=0,pady=(110,0))
rows_entry = Entry((frame))
rows_entry.grid(row=0,column=1,pady=(110,0))
cols_label.grid(row=1,column=0)
cols_entry = Entry((frame))
cols_entry.grid(row=1,column=1)
rand = True
Walls = []#[(1, 2), (1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
player = []
end = []
def custom_grid():
    global rand
    rand = False
    root.destroy()
def next_command():
    global cols, rows
    cols = int(cols_entry.get())
    rows = int(rows_entry.get())

    # clear frame
    for widgets in frame.winfo_children():
      widgets.destroy()
    rand_button = Button(frame, text = "Random Grid",width = 15, height = 2,borderwidth=4, command= root.destroy)
    custom_button = Button(frame, text = "Custom Grid",width = 15, height = 2,borderwidth=4, command= custom_grid)
    rand_button.grid(row=0,column=0,pady=110, padx=10)
    custom_button.grid(row=0,column=1,padx=10)


next_button = Button(frame, text = "Next",width = 15, height = 1, command= next_command )
next_button.grid(column=1,pady = 20)
root.mainloop()

In [ ]:

class BT():
    def __init__(self,i,j):
        self.i = i
        self.j = j
        self.btn = Button(window,bg = 'white',width=4,height=2,command=self.wall_btn)
        self.btn.grid(row=i,column=j)
    def wall_btn(self):
        if self.btn['bg'] == 'white':
            self.btn.configure(bg="Black")
            Walls.append((self.j, self.i))

        elif self.btn['bg'] == 'Black':
            self.btn.configure(bg="Blue")
            global player,end
            player.append((self.j,self.i))
            Walls.remove((self.j, self.i))

        elif self.btn['bg'] == 'Blue':
            self.btn.configure(bg="Green")
            end.append((self.j,self.i))
            player.remove((self.j,self.i))

        else:
            end.remove((self.j,self.i))
            self.btn.configure(bg="white")

if rand:       
    player = [(random.randint(0,cols-1),random.randint(0,rows-1))]
    end = [(random.randint(0,cols-1),random.randint(0,rows-1))]
    
if not rand:
    window = Tk()
    window.title("PA1")
    Width = 500/rows
    
        
    def render_grid():
        global rows, cols, Width, player, end
        for i in range(cols):
            for j in range(rows):
                BT(i,j)
                
                
    # render a grid of buttons to customize the graph.
    render_grid()

    def start_game():
        window.mainloop()
    start_game()

In [ ]:
algselect = Tk()
algselect.geometry('300x300')
algselect.title("PA1")
global selection
def set_selection(select):
    global selection
    selection = select
    algselect.destroy()
a_star_btn = Button(algselect,text = "A star",width = 15, height = 1, command=lambda: set_selection("a star"))
a_star_steps_btn = Button(algselect,text = "A star step by step",width = 15, height = 1,command=lambda: set_selection("a star steps"))
genetic_button = Button(algselect,text = "Genetic",width = 15, height = 1,command=lambda: set_selection("genetic"))
theta_button = Button(algselect,text = "Theta star",width = 15, height = 1,command=lambda: set_selection("theta"))
a_star_btn.grid(row= 0, column=0, pady=(40,0), padx = 20)
a_star_steps_btn.grid(row= 0, column=1, pady=(40,0))
genetic_button.grid(row= 1, column=0, pady=(40,0), padx = 20)
theta_button.grid(row= 1, column=1, pady=(40,0), padx = 20)
algselect.mainloop()


In [ ]:
window = Tk()
window.title("PA1")
if rand:
    for i in range(cols):
        for j in range(rows):
            if random.randint(0,9)<2 and (i,j) not in player and (i,j) not in end:
                Walls.append((i,j))

Width = 500/rows
platform = Canvas(window, width = cols*Width, height = rows*Width)
platform.pack()

# coppied from github
class PriorityQueue:
    def __init__(self):
        self.elements = []

    def isEmpty(self):
        return len(self.elements) == 0

    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))

    def get(self):
        return heapq.heappop(self.elements)[1]
    def remove(self,item):
        index = 0
        for i in self.elements:
            if i[1] == item:
                index = i
                break
        self.elements[index] = self.elements[-1]
        self.elements.pop()
        heapq.heapify(self.elements)
    def exist(self,item):
        for i in self.elements:
            if i[1] == item:
               return True
        return False


# return a list containing neighboring squares in the grid 
def get_neighbors(i):
    (x,y) = i
    neighbors = [(x+1,y),(x,y+1),(x-1,y),(x,y-1)]
    for (i,j) in neighbors:
        if i > rows-1 or i<0 or j > cols-1 or j < 0 or (i,j) in Walls :
            neighbors.remove((i,j))
    return neighbors

# returns manhatten distance since we cannot travel diagonally 
def get_heuristic(i,j):
    (x1,y1) = i
    (x2,y2) = j
    return abs(x1-x2)+ abs(y1-y2)

def a_star():
    unexplored = PriorityQueue() # priority queue used to determine next move
    prev_square = {}   #stors the node before current for each node in the path
    f_of_square = {}    # stors the total cost of the path for each node
    explored =[player[0]]
    unexplored.put(player[0],0)
    prev_square[player[0]] = None
    f_of_square[player[0]] = get_heuristic(player[0], end[0])

    while not unexplored.isEmpty():
        current = unexplored.get()
        explored.append(current)
        if current == end[0]:
            break # goal reached
        neighbors = get_neighbors(current)
        for neighbor in neighbors:
            totalCost = f_of_square[current] +1 # g(n)
            f = totalCost + get_heuristic(neighbor, end[0]) # f(n) = g(n)+h(n)
            if (neighbor not in f_of_square or f < f_of_square[neighbor]) and neighbor not in Walls :
                unexplored.put(neighbor, f)
                f_of_square[neighbor] = f
                prev_square[neighbor] = current
    current = end[0]
    path= [] # store the final path 
    while current != player[0]:
        path.append(current)
        current = prev_square[current]
    path.append(player[0])
    path.reverse()
    return path,explored

def run(genome):
    genome_end = [player[0][0],player[0][1]] # stores the stoping point of the genome
    for i in genome:
        if i == 0 :   # down
            genome_end  = [genome_end[0],genome_end[1]+1]
        elif i == 11: # up
            genome_end  = [genome_end[0],genome_end[1]-1]
        elif i == 10: # left
            genome_end  = [genome_end[0]-1,genome_end[1]]
            
        elif i == 1:  # right
            genome_end  = [genome_end[0]+1,genome_end[1]]
        elif i == 2:  # stay
            genome_end  = [genome_end[0],genome_end[1]]
    return (genome_end[0],genome_end[1])

def decrypt_genetic(path_genome):
    path = [player[0]]
    for i in path_genome:
        #0, 11, 2, 10, 1, 0, 0, 2, 0, 10, 1, 2, 2
        if i == 0 :   # down
            path.append((path[-1][0],path[-1][1]+1))
        elif i == 11: # up
            path.append((path[-1][0],path[-1][1]-1))
        elif i == 10: # left
            path.append((path[-1][0]-1,path[-1][1]))
        elif i == 1:  # right
            path.append((path[-1][0]+1,path[-1][1]))
    return path

def fitness(genome):
    end_point = run(genome)
    genome_path = decrypt_genetic(genome)
    j = 0
    for i in genome_path:
        choises = [10,1,0,11,2]
        if (i[0],i[1]) in Walls or i[0]<0 or i[0]>cols-1 or i[1]<0 or i[1]>rows-1:
            if type(genome[j-1]) is list:
                genome[j-1] = genome[j-1][0]
            choises.remove(genome[j-1])
            genome[j-1] = random.choices(choises)[0]
            end_point = run(genome)
            genome_path = decrypt_genetic(genome)
        if(j<len(genome)-1):
            j+=1
    if end_point == end[0]:
        return 0
    return -get_heuristic(end_point,end[0]) # returnss the distance from the goal.

def Genetic_algorithm():
    choises = [10,1,0,11,2]
    solutions = []
    genome_size = int(1.3*rows)
    for i in range(100):
        genome = []
        genomelist = [random.choices(choises) for i in range(genome_size)]
        for i in range(len(genomelist)):              
            genome.append(genomelist[i][0])
        solutions.append(genome)
    for i in range (1000):
        rankedsolutions = []
        for j in solutions:
            rankedsolutions.append((fitness(j),j))
        rankedsolutions.sort(reverse= True)
        print(f'=== Gen {i} best solutions ===')
        print(rankedsolutions[0])
        
        if rankedsolutions[0][0] == 0:
            break
        bestsolutions = rankedsolutions[:100]
        newGen = []
        splice_range = round(0.4*genome_size)
        for i in range(0,100,2):
            sol1 = bestsolutions[i][1]
            sol2 = bestsolutions[i+1][1]
            child1 = sol1[:-splice_range]+sol2[-splice_range:]
            child2 = sol2[:-splice_range]+sol1[-splice_range:]
            newGen.append(child1)
            newGen.append(child2)
        solutions = newGen
    return rankedsolutions[0][1]

def theta_star():
    open = PriorityQueue() # priority queue used to determine next move
    parent = {}   #stors the node before current for each node in the path
    gScore = {}    # stors the total cost of the path for each node
    gScore[player[0]] = 0
    parent[player[0]] = player[0]
    open.put(player[0],gScore[player[0]]+get_heuristic(player[0],end[0]))
    closed = []
    def line_of_sight(node,target):
        if node == None:
            return False
        startx,starty = node[0],node[1]
        endx,endy = target[0],target[1]
        if startx == endx: # on the same row
            if starty > endy:  # start is below end
                for i in range(starty-endy):
                    if (startx,starty-i) in Walls:
                        return False
                return True
            else:  # start is above end
                for i in range(endy-starty):
                    if (startx,starty+i) in Walls:
                        return False
                return True
        if starty == endy: # on the same column
            if startx > endx:  # start is to the right of end
                for i in range(startx-endx):
                    if (startx-i,starty) in Walls:
                        return False
                return True
            else:  # start is to the left of end
                for i in range(endx-startx):
                    if (startx+i,starty) in Walls:
                        return False
                return True
        else:
            return False
    def euclidean(node,target):
        startx,starty = node[0],node[1]
        endx,endy = target[0],target[1]
        return sqrt((startx-endx)**2+(starty-endy)**2)
    def update_vertex(s,neighbor):
        if line_of_sight(parent[s],neighbor):
            if gScore[parent[s]]+euclidean(parent[s],neighbor) < gScore[neighbor]:
                gScore[neighbor] = gScore[parent[s]]+euclidean(parent[s],neighbor)
                parent[neighbor] = parent[s]
                if open.exist(neighbor):
                    open.remove(neighbor)
                open.put(neighbor,gScore[neighbor]+get_heuristic(neighbor,end[0]))
        else:
            if gScore[s] + euclidean(s,neighbor) < gScore[neighbor]:
                gScore[neighbor] = gScore[s] + euclidean(s, neighbor)
                parent[neighbor] = s 
                if open.exist(neighbor):
                    open.remove(neighbor)
                open.put(neighbor,gScore[neighbor]+get_heuristic(neighbor,end[0]))
    while  not open.isEmpty():
        s   = open.get()

        if s == end[0]:
            break
        closed.append(s)
        neighbors = get_neighbors(s)
        for i in neighbors:
            if i in Walls:
                neighbors.remove(i)
        for neighbor in neighbors:
            if neighbor not in closed:
                if not open.exist(neighbor):
                    gScore[neighbor] = 10000
                    parent[neighbor] = None
                    update_vertex(s, neighbor)
    


    current = end[0]
    path= [] # store the final path 
    while parent[current] != player[0]:
        path.append(current)
        current = parent[current]
    path.append(player[0])
    path.reverse()
    return path



def render_grid():
    global rows, cols, Width, player, end
    for i in range(cols):
        for j in range(rows):
            platform.create_rectangle(i*Width, j*Width,(i+1)*Width, (j+1)*Width, fill="white", width = 1)

    platform.create_rectangle(player[0][0]*Width, player[0][1]*Width,(player[0][0]+1)*Width, (player[0][1]+1)*Width, fill="blue", width = 1)
    platform.create_rectangle(end[0][0]*Width, end[0][1]*Width,(end[0][0]+1)*Width, (end[0][1]+1)*Width, fill="green", width = 1)

    for (i,j) in Walls:
        platform.create_rectangle(i*Width, j*Width,(i+1)*Width, (j+1)*Width, fill="black", width = 1)

def render_path():
    global Width,path
    for (i,j) in path:
        platform.create_rectangle(i*Width, j*Width,(i+1)*Width, (j+1)*Width, fill="Red", width = 1)
        time.sleep(0.1)
render_grid()
global rendered 
rendered= False
def render_path_steps():
    global Width,steps,rendered
    if len(steps) != 0:
        (i,j) = steps[0]
        steps.remove((i,j))
        platform.create_rectangle(i*Width, j*Width,(i+1)*Width, (j+1)*Width, fill="blue", width = 1)
    if (len(steps)==0 and not rendered):
        render_path()
        rendered = True

def start_game():
    window.mainloop()
global path,steps
if selection == 'a star':
    path,steps= a_star()
    render_path()
elif selection == 'a star steps':
    path,steps = a_star()
    next_step_button = Button(window,text = "Next Step",width = 15, height = 2, pady = 10,command = render_path_steps)
    next_step_button.pack()
 
elif selection == "genetic":
   path_genome= Genetic_algorithm()
   path = decrypt_genetic(path_genome)
   render_path()
elif selection == "theta":
   path = theta_star()
   render_path()
start_game()